<a href="https://www.kaggle.com/code/yaaangzhou/pg-s3-e21-very-simple-approach?scriptVersionId=145414858" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on public kernal. thanks to all for your efforts.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e21/sample_submission.csv')

In [3]:
target = 'target'
num_var = df.columns.drop(['target','id'])

In [4]:
df = df[df.id != 2365]
df.target = df.target.clip(7, 20)

In [5]:
# SGD
clf = SGDOneClassSVM(nu=0.55).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(clf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# IsolationForest
lf = IsolationForest(random_state=0).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(lf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# LOF
lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
predictions = lof.fit_predict(df[num_var])
df = df.drop(index=np.where(predictions == -1)[0])
df.reset_index(drop=True, inplace=True)

# Tricks from https://www.kaggle.com/code/warcoder/lb-1-32253-lof-svm-iforest-cleanlab
lowest_quality_labels = [2294,448, 437,309,1684,773,1722,2007]
df.drop(list(lowest_quality_labels),inplace=True)

In [6]:
drop_index = np.random.choice(df[df['target']==7].index.to_numpy(), 130, replace=False)
df.loc[drop_index]

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
1761,2016,7.0,4.90,8.90,8.980,9.265,9.265,9.805,7.010,0.14,...,8.430,1.730,1.800,4.20,4.500,3.950,3.725,8.415,4.250,3.70
1374,1571,7.0,9.30,7.30,10.433,9.500,6.360,9.440,8.367,0.20,...,3.660,2.420,1.830,4.20,3.267,3.800,6.367,6.450,3.950,3.80
2710,3084,7.0,5.04,5.20,9.265,7.795,9.805,8.765,7.805,0.20,...,8.805,1.285,0.636,5.40,5.700,7.625,6.485,7.695,2.210,1.83
1549,1763,7.0,7.10,5.60,9.260,7.170,5.010,8.980,5.970,0.49,...,4.100,1.640,2.850,7.80,8.000,2.900,6.950,8.720,5.540,6.34
2671,3041,7.0,5.61,4.98,13.700,13.700,6.015,10.070,6.375,0.24,...,7.795,1.730,1.730,2.52,2.910,8.465,9.550,26.500,8.250,10.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,3397,7.0,5.89,8.20,9.545,7.600,1.590,9.805,7.150,0.24,...,14.775,4.320,1.800,7.60,8.000,2.070,6.075,6.750,3.500,3.80
1637,1866,7.0,2.60,3.30,8.100,6.550,6.015,10.070,7.360,0.12,...,21.070,0.644,0.636,5.30,4.300,4.990,5.600,7.695,5.520,5.70
2166,2471,7.0,3.30,7.40,12.020,6.830,4.550,12.030,3.150,0.36,...,0.680,2.650,4.720,6.30,4.800,3.950,6.550,6.750,1.950,2.60
962,1103,7.0,5.89,9.00,8.770,5.240,4.660,8.430,7.150,0.24,...,7.795,0.636,1.800,3.15,2.520,4.725,6.075,6.750,4.725,4.14


In [7]:
col_to_keep = ['O2_1', 'O2_2', 'BOD5_5', 'target']

for i in df.columns:
    if i not in col_to_keep:
        df[i]=0

In [8]:
df.to_csv('submission.csv',index=False)
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,0,9.10,13.533,40.900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,0,8.21,3.710,5.420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,0,8.39,8.700,8.100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,0,8.07,8.050,8.650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,0,8.65,8.367,8.760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3073,0,8.08,6.250,8.300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.695,0,0
3074,0,8.09,6.630,6.630,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3075,0,9.95,8.367,8.433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
